In [1]:
import transformers
import pandas as pd
import json
import os

In [2]:
outerFolder = "/shared/3/projects/benlitterer/agendaSetting/testingHtml"

dfList = []
for folderNum in os.listdir(outerFolder): 
    innerFolder = outerFolder + "/" + folderNum 
    for innerFileName in os.listdir(innerFolder): 
        innerPathName = outerFolder + "/" + folderNum + "/" + innerFileName
        if "json" in innerPathName: 
            with open(innerPathName, "r") as f: 
                data = json.loads(f.read())
            df = pd.json_normalize(data)
            df["id"] = innerFileName.split(".")[0]
            dfList.append(df)
        

In [3]:
fullDf = pd.concat(dfList).reset_index(drop=True)

In [4]:
leanDf = fullDf[["id", "title", "url", "source_url", "text"]]

In [5]:
leanDf

,id,title,url,source_url,text
0,1484189203,Police: 2 men stole tools from Lowe’s in Davie,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com,"DAVIE, FLA. (WSVN) - Police need help catching..."
1,1484261803,Woody Guthrie's 1943 New Year's Resolutions ar...,https://boingboing.net/2020/01/01/woody-guthri...,https://boingboing.net,Woody Guthrie's 1943 New Year's Resolutions ar...
2,1484084603,Thanos' Defeat Leaves The Door Open For Bigger...,https://screenrant.com/thanos-defeat-new-mcu-p...,https://screenrant.com,Thanos was finally dispatched (twice!) over th...
3,1510471003,India police rescue 23 kids held hostage,https://www.news.com.au/world/breaking-news/in...,https://www.news.com.au,Authorities in northern India have rescued 23 ...
4,1513643803,СБУ опровергла связь обысков в Киеве с деятель...,https://www.interfax.ru/world/694220,https://www.interfax.ru,СБУ опровергла связь обысков в Киеве с деятель...
...,...,...,...,...,...
9327,1536301236,Supreme Court to decide fate of Obamacare,https://web.archive.org/web/20201204113047id_/...,https://web.archive.org,Hide Transcript Show Transcript\n\na new poll ...
9328,1577188836,Cuenta Chihuahua con capacidad para realizar 5...,https://web.archive.org/web/20200427090226id_/...,https://web.archive.org,El gobernador de Chihuahua anunció que se cuen...
9329,1549683936,Torra da positivo - El Digital de Albacete,https://web.archive.org/web/20200602152016id_/...,https://web.archive.org,/Redacción/\n\nEl presidente de la Generalitat...
9330,1544189636,Coronavirus: River pide a sus socios que no va...,https://web.archive.org/web/20200408005939id_/...,https://web.archive.org,Autoridades del club “millonario” emitieron un...


In [6]:
groundTruths = pd.read_csv("/shared/3/projects/benlitterer/agendaSetting/final_evaluation_data.csv")
print(groundTruths.shape)
groundTruths.head(3)

(4902, 14)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667


In [7]:
#get just the english pairs 
groundTruths.loc[(groundTruths["url1_lang"] == "en") & (groundTruths["url2_lang"] == "en")].shape

(236, 14)

In [19]:
def splitIds(inStr): 
    return inStr.split("_")
#groundTruths[["id1", "id2"]] = groundTruths["pair_id"].apply(splitIds)
groundTruths["id1"] = [item[0] for item in groundTruths["pair_id"].apply(splitIds)]
groundTruths["id2"] = [item[1] for item in groundTruths["pair_id"].apply(splitIds)]

In [20]:
#This used to work but stopped working potentially due to pandas version error? 
#groundTruths[["id1", "id2"]] = groundTruths["pair_id"].str.split("_", expand=True)

In [21]:
leanDf = leanDf[["id", "text"]]

In [22]:
#merge text for the first id, id1
leanDf1 = leanDf.rename(columns = {"id":"id1","text":"text1"})
firstMerge = pd.merge(groundTruths, leanDf1, how="left", on="id1")
firstMerge.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE,id1,id2,text1
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000,1484189203,1484121193,"DAVIE, FLA. (WSVN) - Police need help catching..."
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000,1484011097,1484011106,Govt officials confirm Trump can block US comp...
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667,1484039488,1484261803,"Ducks defenseman Erik Gudbranson, left, knocks..."


In [23]:
leanDf2 = leanDf.rename(columns = {"id":"id2","text":"text2"})
merged = pd.merge(firstMerge, leanDf2, how="left", on="id2")
merged.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE,id1,id2,text1,text2
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000,1484189203,1484121193,"DAVIE, FLA. (WSVN) - Police need help catching...","DEERFIELD BEACH, FLA. (WSVN) - A no-swim advis..."
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000,1484011097,1484011106,Govt officials confirm Trump can block US comp...,The latest huge unsecured cloud storage find i...
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667,1484039488,1484261803,"Ducks defenseman Erik Gudbranson, left, knocks...",Woody Guthrie's 1943 New Year's Resolutions ar...


In [24]:
mergedFull = merged.dropna(subset=["text1", "text2"])

In [25]:
mergedFull.shape

(4689, 18)

In [27]:
mergedEn = mergedFull[(mergedFull["url1_lang"] == "en") & (mergedFull["url2_lang"] == "en")]
mergedEn.shape

(234, 18)

In [28]:
mergedEn.to_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/enTestData.csv", sep="\t")